# Averaged Artifact Substraction with EEG Data
## First step prepare Jupyter Notebook

In [ ]:
import os
from facet.facet import facet
from loguru import logger
import sys


eegDataSet_filepath = "../datasets/NiazyFMRI.edf"
motiondata_moosmann_filepath = "../datasets/headmotiondata_209s.tsv"
export_bids_path = "./bids_dir"

# logging configuration
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add("facet.log", level="DEBUG")

# defining important parameters
window_size = 30
upsampling_factor = 10
artifact_to_trigger_offset_seconds = -0.005
relative_window_position = -0.5
moosmann_motion_threshold = 0.8
event_id_description_pairs = {"trigger": 1}
regex_trigger_annotation_filter = (
    r"\b1\b"  # Annotations with the description 'trigger' are considered as triggers
)
regex_trigger_event_filter = (
    r"\b1\b"  # Events with the value 1 are considered as triggers
)
unwanted_bad_channels = [
    "EKG",
    "EMG",
    "EOG",
    "ECG",
]  # Channels with these names are considered as bad channels and not considered in the processing
evaluation_measures = ["SNR", "RMS", "RMS2", "MEDIAN"]

f = facet()

## Second Step import EEG Data

In [ ]:
f.import_eeg(
    eegDataSet_filepath,
    artifact_to_trigger_offset=artifact_to_trigger_offset_seconds,
    upsampling_factor=upsampling_factor,
    bads=unwanted_bad_channels,
)
f.plot_eeg(title="after import", start=29)

## Third Step Preprocessing

In [ ]:
f.get_eeg().mne_raw.crop(0, 162)  # Crop because the edf has bad data samples at the end
f.pre_processing()

## Fourth Step find triggers

In [ ]:
f.find_triggers(regex_trigger_annotation_filter)  # Using Niazys data
f.align_triggers(0)
f.get_analysis().print_analytics()

## Fourth step apply AAS

In [ ]:
f.calc_matrix_aas(window_size=window_size, rel_window_position=relative_window_position)

## Fifth step Remove Artifacts

In [ ]:
f.remove_artifacts()

## Sixth step Post-Processing

In [ ]:
f.post_processing()

## Seventh display the processed eeg data

In [ ]:
f.plot_eeg(title="after lowpass", start=29)

## Eighth Evaluate Results

In [ ]:
results_numpy_matrix= f.evaluate(f.get_eeg(), name="numpy_matrix", measures = evaluation_measures)
# apply moosmann
f.import_eeg(
    eegDataSet_filepath,
    artifact_to_trigger_offset=artifact_to_trigger_offset_seconds,
    bads=unwanted_bad_channels,
)
f.get_eeg().mne_raw.crop(0, 162)  # Crop because the edf has bad data samples at the end
f.pre_processing()
f.find_triggers(regex_trigger_event_filter)
f.align_triggers(0)
f.calc_matrix_motion(
    file_path=motiondata_moosmann_filepath, threshold=moosmann_motion_threshold
)
f.remove_artifacts()
f.post_processing()
results_moosmann = f.evaluate(f.get_eeg(), name="Moosmann", measures = evaluation_measures)

f.plot([results_numpy_matrix, results_moosmann], plot_measures=evaluation_measures)

f.export_eeg(event_id=event_id_description_pairs, path=export_bids_path, fmt="bids")